<a href="https://colab.research.google.com/github/MadKauSing/GuessFlat/blob/main/GuessFlat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MADKAUSING GUESSFLAT
```
Ayush Singh          -PES2UG20C0S81
Ayushmaan Kaushik    -PES2UG20CS082
Bhavini Madhuranath  -PES2UG20CS088
```

### An indian music genre classifier


In [ ]:
#linking google drive
from google.colab import drive
drive.mount('/content/drive')


---
**Importing the dataset and storing file names in a pandas dataframe**

Please follow the given format

|file_path|genre|
|----|----|
|path_to_file1.mp3|genre1|
|path_to_file2.mp3|genre2|
|path_to_file3.mp3|genre3|

---


In [ ]:
import os
import pandas as pd
import matplotlib
import librosa
import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
import librosa.display

In [ ]:
os.getcwd()

In [ ]:
moozyc_path='/content/drive/MyDrive/moozyc'
genre_list=os.listdir(moozyc_path)
#genre_list

In [ ]:
data=[]
for genre in genre_list:
  genre_path='/'+genre
  music_files=os.listdir(moozyc_path+genre_path)
  music_files=[i  for i in music_files if i!='.spotdl-cache']
  
  #add each song to pandas df
  for song in music_files:
    song_path=moozyc_path+genre_path+'/'+song
    data.append([song_path,song,genre])

#creating pandas dataframe
song_df=pd.DataFrame(data,columns=['file_path','song','genre'])
song_df

---
** Reading mp3 files and generating spectrogram **

---

In [ ]:
%rm -rf /content/spectrograms6sec

In [ ]:
os.mkdir('/content/spectrograms6sec')
for genre in genre_list:
  os.mkdir(f'/content/spectrograms6sec/{genre}')

In [ ]:
#making spectrograms save in file
def save_spectrogram(block,sr,genre,song_name,counter):
  matplotlib.use('Agg')
  S = librosa.feature.melspectrogram(y=block, sr=sr, n_mels=128,fmax=8000)
  plt.figure(figsize=(6, 3))
  librosa.display.specshow(librosa.power_to_db(S,ref=np.max),y_axis='mel', fmax=8000,x_axis='time')
  # plt.plot()
  song_name=song_name.replace('.mp3','')
  file_name=f'/content/spectrograms6sec/{genre}/'+song_name+str(counter)+'.png'
  plt.savefig(file_name)



In [ ]:

def preprocess(file_name,song_name,genre):
  # First load the file
  audio, sr = librosa.load(file_name)
  filename = file_name.replace(".mp3",".wav")

  # Get number of samples for 6 seconds; replace 2 by any number
  buffer = 6 * sr

  samples_total = len(audio)
  samples_wrote = 0
  counter = 1

  while samples_wrote < samples_total:
      
      #check if the buffer is not exceeding total samples 
      if buffer > (samples_total - samples_wrote):
          buffer = samples_total - samples_wrote

      block = audio[samples_wrote : (samples_wrote + buffer)]
      out_filename = "split_" + str(counter) + "_" + filename

      # Write 2 second segment
      save_spectrogram(block,sr,genre,song_name,counter)
      # sf.write('1.wav', block, sr, 'PCM_24')
      counter += 1
      samples_wrote += buffer
      
  
    

In [ ]:
# preprocess(song_df["file_path"][0],song_df['song'][0],song_df['genre'][0])
# song_df["file_path"][0]

In [ ]:
for index,row in song_df.iterrows():
  preprocess(row["file_path"],row["song"],row["genre"])
  print(f"{row['song']} done")

In [ ]:
%ls